# Show resource allocations and logs on a plot

**Note:** you must install `google-cloud-monitoring` to use this, otherwise, comment out the import monitoring_v3 and fetch_memory_usage lines and set mem_time and mem_y to None in plot_memory_usage

In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from google.cloud import monitoring_v3
from google.protobuf import duration_pb2
import sys
from pathlib import Path
from scipy.interpolate import interp1d

sys.path.append(str(Path.cwd().parent / 'cerulean_cloud') + "/")
from structured_logger import (
    log_query,
    query_logger,
    generate_log_file,
    get_scene_log_stats,
    get_latest_revision,
)

project_id = "cerulean-338116"
service_name = "cerulean-cloud-test-cr-orchestrator"
revision_name = "cerulean-cloud-test-cr-orchestrator-00119-pw6"
# revision_name = get_latest_revision(project_id, service_name)

now = datetime.datetime.now()
end_time = now+datetime.timedelta(days=1)
start_time = now - datetime.timedelta(days=2)  # N days ago
# end_time = datetime.datetime(2025,2,8,0,0)
# start_time = datetime.datetime(2025,2,5,0,0)
tz_local = "US/Arizona"

project_name = f"projects/{project_id}"
    

In [ ]:


def fetch_memory_usage(revision_name, start_time, end_time, interval_min=5):
    """
    Fetch Cloud Run memory usage over the last 3 days.
    """
    metric_type = "run.googleapis.com/container/memory/utilizations"
    
    # Define query filter
    filt = f'''
        metric.type = "{metric_type}"
        AND resource.labels.revision_name = "{revision_name}"
    '''
    
    # Define aggregation settings
    aggregation=monitoring_v3.Aggregation(
        alignment_period=duration_pb2.Duration(seconds=interval_min*60),
        per_series_aligner=monitoring_v3.Aggregation.Aligner.ALIGN_PERCENTILE_50 
    )

    # Define query request
    query = monitoring_v3.ListTimeSeriesRequest(
        name=project_name,
        filter=filt,
        interval=monitoring_v3.TimeInterval(start_time=start_time, end_time=end_time),
        aggregation=aggregation,
        view=monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    )

    try:
        # Initialize Cloud Monitoring client
        client = monitoring_v3.MetricServiceClient()
        results = client.list_time_series(query)
    except Exception as e:
        print(f"Error fetching memory usage for revision {revision_name}: {e}")
        return pd.DataFrame(columns=["timestamp", "memory_usage"])  # Return empty dataframe

    # Extract data points
    data = []
    for result in results:
        for point in result.points:
            timestamp = point.interval.end_time
            value = point.value.double_value if hasattr(point.value, "double_value") else None
            data.append([timestamp, value])

    return pd.DataFrame(data, columns=["timestamp", "memory_usage"])



def interp_logs_to_mem_ts(mem_ts, log_ts, mem):
    # Create interpolation function
    interp_func = interp1d(
        mem_ts.astype(np.int64) / 10**9,  # Convert timestamps to seconds
        mem,
        kind="linear", 
        fill_value="extrapolate" 
    )
    
    return interp_func(log_ts.astype(np.int64) / 10**9)


def add_trace_hover(fig, df, size, color, name):

    
    fig.add_trace(go.Scatter(
        x=df['timestamp'], 
        y=df['y'], 
        mode='markers', 
        marker=dict(size=size, color=color),
        text=df['text'], 
        name=name,
        hoverinfo='text',
    ))

    return fig


def add_severity_trace(fig, logs):

    severities = ["INFO","WARNING","ERROR", "DEBUG"]
    colors = ["blue", "orange", "red", "green"]
    sizes = [3, 5, 10]

    
    tmp = logs[logs['mem_usage'].isnull()==False]
    fig.add_trace(go.Scatter(
        x=tmp['timestamp'], 
        y=tmp['y'], 
        mode='lines',
        line=dict(color='gray', width=1, dash='dash'),
        name="Memory Usage",
        hoverinfo='skip' 
    ))

    for severity, color, size in zip(severities, colors, sizes):

        tmp = logs[logs['severity']==severity]
        fig = add_trace_hover(fig, tmp, size, color, name=severity)
        
    return fig

def add_initiating_orchestrator_trace(fig, logs, size=5, color="green"):
    tmp = logs[logs['message'].str.contains("initiating orchestrator",case=False, na=False)]
    fig = add_trace_hover(fig, tmp, size, color, name="orchestrator initiaion")

    return fig


def plot_memory_usage(mem_time=None, mem_y=None, logs=None, plot_type="matplotlib", label="Memory Usage (%)"):
    if plot_type == "matplotlib":
        plt.figure(figsize=(10, 5))
        plt.plot(df_mem["timestamp"].dt.tz_convert("US/Arizona"), df_mem["memory_usage"], label=label)
        plt.xlabel("Timestamp")
        plt.ylabel("Memory Usage (bytes)")
        plt.title(f"Cloud Run Memory Usage for {revision_name}")
        plt.xticks(rotation=45)
        plt.legend()
        plt.show()
    elif plot_type =="plotly":
        # Create Scatter Plot with Clickable Points
        fig = go.Figure()

        if mem_time is not None:
            fig.add_trace(go.Scatter(
                x=mem_time, 
                y=mem_y, 
                mode='lines',
                line=dict(color='black', width=2, dash='solid'), 
                name="Memory Usage",
                hoverinfo='skip' 
            ))
        

        if logs is not None:
            fig = add_severity_trace(fig, logs)
            fig = add_initiating_orchestrator_trace(fig, logs, size=8, color="cyan")
            
        
        fig.update_layout(
            title=label,
            hovermode="closest",
            dragmode="zoom"
        )
        
        fig.show(config={"scrollZoom": True})

        return fig




# Plot memory usage and logs

**Note:** memory allocation seems to be offset a bit from logs, so to track memory usage, it is best to ensure you are tracking `mem_usage` with the logger, which can be done by setting `track_memory_usage=True` in structured_logger.py<br><br>
**Note:** Some of the internal ERRORs (not the ones manually set in the codebase) do not seem to be lined up in time. An end of life event often appears long before the actual event. These will show up on the plot as red points with y=0 on the plot

In [ ]:

print("querying logger")
query = log_query(
    service_name,
    revision_name=revision_name,
    start_time=start_time,
    end_time=end_time,
)
logs = query_logger(project_id, query)
logs = logs.sort_values("timestamp").reset_index(drop=True)
logs["timestamp"] = logs["timestamp"].dt.tz_convert(tz_local)
logs["mem_usage"] = logs['json_payload'].apply(lambda x: x['perc_ram_used']/100 if x is not None and 'perc_ram_used' in x else None)
instance_id = logs['instanceId'].unique()[0]

print("querying metrics")
df_mem = fetch_memory_usage(revision_name, start_time, end_time, interval_min=1)
df_mem["timestamp"] = df_mem["timestamp"].dt.tz_convert(tz_local)

if len(logs['mem_usage'].dropna()) > 0:
    logs['y'] = logs['mem_usage'].fillna(0)
else:
    logs['y'] = interp_logs_to_mem_ts(df_mem['timestamp'], logs['timestamp'], df_mem["memory_usage"])


# Plot Memory Usage
if not df_mem.empty:
    logs0 = logs.copy()
    logs0['text'] = logs0['message']
    fig = plot_memory_usage(mem_time=df_mem["timestamp"], mem_y=df_mem["memory_usage"], logs=logs0, plot_type="plotly")
else:
    print(f"No memory usage data found for revision {revision_name}")


In [ ]:
generate_log_file(logs, filename="log.txt")